<a href="https://colab.research.google.com/github/rohitgambhir/MACHINE-LEARNING-SIGNATURE/blob/main/SignatureAssignmentusingResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')









import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import cv2
import math

import keras
import keras.backend as K
import tensorflow as tf
from keras import applications
from keras.models import Model
from keras.layers import Flatten, Dense, Input,concatenate
from keras.optimizers import Adam
from keras.models import load_model, model_from_json
import random

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import cv2
import math
from tensorflow import keras
import keras
import keras.backend as K
import tensorflow as tf
from keras import applications
from keras.models import Model
from keras.layers import Flatten, Dense, Input,concatenate
from keras.optimizers import Adam
from keras.models import load_model, model_from_json
import random
gen="/content/drive/MyDrive/input/handwritten signatures/sample_Signature/sample_Signature/genuine"
forg="/content/drive/MyDrive/input/handwritten signatures/sample_Signature/sample_Signature/forged"

gentr="/content/drive/MyDrive/input/sigcomp 2009 train/Sigcomp 2009 train/genuine"
forgtr="/content/drive/MyDrive/input/sigcomp 2009 train/Sigcomp 2009 train/forgeries"

gent="/content/drive/MyDrive/input/Sigcomp 2009/sigcomp 2009/genuines"
forgt="/content/drive/MyDrive/input/Sigcomp 2009/sigcomp 2009/forgeries"

In [ ]:
img_width, img_height, channels = 224, 224, 3

dim = (img_width, img_height)

def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

def returnimages(path,img):
    image=cv2.imread(path+"/"+ img)                  #bringing the image
    image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=to_rgb(image).reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image

def getfiles(num,gen,forg):
    a=os.listdir(gen)
    b=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    n,m=[],[]
    for i in b:
        if i.endswith(c+".png"):
            n=n+[i]
        elif i.endswith(c+".PNG"):
            n=n+[i]
    for i in a:
        if i.endswith(c+".png"):
            m=m+[i]
        elif i.endswith(c+".PNG"):
            m=m+[i]
    return m.pop(),n,m

def getfiles2(num):
    a=os.listdir(gentr)
    b=os.listdir(forgtr)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    n,m=[],[]
    for i in b:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            n=n+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            n=n+[i]
    for i in a:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            m=m+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            m=m+[i]
    return m.pop(),n,m


In [ ]:
def triplet_loss(y_true, y_pred):
    alpha = 0.5
    anchor, positive, negative =y_pred[0,0:512], y_pred[0,512:1024], y_pred[0,1024:1536]
    
    positive_distance = K.mean(K.square(anchor - positive),axis=-1)
    negative_distance = K.mean(K.square(anchor - negative),axis=-1)
    return K.mean(K.maximum(0.0, positive_distance - negative_distance + alpha))



In [ ]:

model = keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='max')
for layer in model.layers[:15]:
    layer.trainable = False

anchor_in = Input(shape=(img_width, img_height, channels))
pos_in = Input(shape=(img_width, img_height, channels))
neg_in = Input(shape=(img_width, img_height, channels))

anchor_out = model(anchor_in)
pos_out = model(pos_in)
neg_out = model(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out],axis=1)

model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

In [ ]:
model.compile(optimizer=Adam(lr=0.000005),loss=triplet_loss)

In [ ]:
def generator():
    for i in range(1,31):
        if(i<10):
            anc,neg,pos=getfiles(float("0.00"+str(i)),gen,forg)
        else:
            anc,neg,pos=getfiles(float("0.0"+str(i)),gen,forg)
        for i in range(len(neg)):
            for j in range(len(pos)):
                anchor=returnimages(gen,anc)
                positive=returnimages(gen,pos[j])
                negative=returnimages(forg,neg[i])
               # yield ([anc,pos[j],neg[i]],[0])
                yield ([anchor,positive,negative],np.asarray([0]))

In [ ]:
for x in range(2):
    model.fit_generator(generator() , steps_per_epoch=200,epochs=3)

In [ ]:
model.compile(optimizer=Adam(lr=0.000002),loss=triplet_loss)

In [ ]:
def generator2():
    x=["0.001","0.004", "0.005", "0.006", "0.007",
       "0.008", "0.009", "0.010", "0.011", "0.001", "0.010"]
  #  x=["0.001", "0.004", "0.006", "0.010"]

    for k in x:
        anc,neg,pos=getfiles2(k)
        frac=0.95    
        inds = set(random.sample(list(range(len(neg))), int(frac*len(neg))))
        neg = [n for i,n in enumerate(neg) if i not in inds]
    
        for i in range(len(neg)):
            for j in range(len(pos)):
                anchor=returnimages(gentr,anc)
                positive=returnimages(gentr,pos[j])
                negative=returnimages(forgtr,neg[i])
               # yield ([anc,pos[j],neg[i]])
                yield ([anchor,positive,negative],np.asarray([0]))

In [ ]:
for x in range(2):
    model.fit_generator(generator2(),steps_per_epoch=32,epochs=11)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/11
32/32 [==============================] - 84s 2s/step - loss: 0.1814
Epoch 2/11
32/32 [==============================] - 68s 2s/step - loss: 0.1571
Epoch 3/11
32/32 [==============================] - 70s 2s/step - loss: 0.1208
Epoch 4/11
32/32 [==============================] - 69s 2s/step - loss: 0.0512
Epoch 5/11
32/32 [==============================] - 70s 2s/step - loss: 0.0000e+00
Epoch 6/11
32/32 [==============================] - 70s 2s/step - loss: 0.1858
Epoch 7/11
32/32 [==============================] - 73s 2s/step - loss: 0.0000e+00
Epoch 8/11
32/32 [==============================] - 70s 2s/step - loss: 0.0000e+00
Epoch 9/11
32/32 [==============================] - 70s 2s/step - loss: 0.0000e+00
Epoch 10/11
32/32 [==============================] - 70s 2s/step - loss: 0.0000e+00
Epoch 11/11
32/32 [==============================] - 70s 2s/step - loss: 0.0000e+00
Epoch 1/11
32/32 [==============================] - 71s 2s/step - loss: 0.0000e+00
Epoch 2/11
32/32 [====

In [23]:
tneg,tpos=0,0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]
for k in x: #the id of signatures you want to check
    #print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    tneg=tneg+len(neg)
    tpos=tpos+len(pos)
print(tneg,tpos)


624 203


In [24]:
model.save_weights('/content/drive/MyDrive/input')

In [30]:
forg_passed=0
gen_flagged=0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    
    anchor=returnimages(gent,anc)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor,positive,anchor])
        a, p, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist=np.linalg.norm(a-p)
        print("positive distance is ",dist)
        if(dist>70):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor,negative,anchor])
        a, n, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist=np.linalg.norm(a-n)
        print("negative distance is ",dist)
        if(dist>62):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)


When k is  0.002
When k is  0.008
When k is  0.016
When k is  0.018
When k is  0.024
When k is  0.033
When k is  0.035
When k is  0.044
When k is  0.046
When k is  0.063
gen flagged -  72.005264 file name is -  NFI-06303063.png
gen flagged -  72.005264 file name is -  NFI-06306063.png
gen flagged -  72.005264 file name is -  NFI-06307063.png
gen flagged -  72.005264 file name is -  NFI-06308063.png
gen flagged -  72.005264 file name is -  NFI-06309063.png
gen flagged -  72.005264 file name is -  NFI-06310063.png
gen flagged -  72.005264 file name is -  NFI-06311063.png
gen flagged -  72.005264 file name is -  NFI-06302063.png
gen flagged -  72.005264 file name is -  NFI-06301063.png
gen flagged -  72.005264 file name is -  NFI-06305063.png
gen flagged -  72.005264 file name is -  NFI-06312063.png
When k is  0.07
gen flagged -  71.61155 file name is -  NFI-07008070.png
gen flagged -  71.61155 file name is -  NFI-07001070.png
gen flagged -  71.61155 file name is -  NFI-07007070.png
gen f

In [31]:
def getfilest(num,gen,forg):
    a=os.listdir(gen)
    b=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    n,m=[],[]
    for i in b:
        if i.endswith(c+".png"):
            n=n+[i]
        elif i.endswith(c+".PNG"):
            n=n+[i]
    for i in a:
        if i.endswith(c+".png"):
            m=m+[i]
        elif i.endswith(c+".PNG"):
            m=m+[i]
    return m.pop(),m.pop(),n,m

In [ ]:
forg_passed=0
gen_flagged=0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc1,anc2,neg,pos=getfilest(k,gent,forgt)
    
    anchor1=returnimages(gent,anc1)
    anchor2=returnimages(gent,anc2)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor1,positive,anchor2])
        a1, p, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist1=np.linalg.norm(a1-p)
        dist2=np.linalg.norm(a2-p)
        dist=(dist1+dist2)/2
        #print("positive distance is ",dist)
        if(dist>66):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor1,negative,anchor2])
        a1, n, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist1=np.linalg.norm(a1-n)
        dist2=np.linalg.norm(a2-n)
        #print("negative distance is ",dist)
        dist=(dist1+dist2)/2
        if(dist>65):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)


When k is  0.002
gen flagged -  66.09561920166016 file name is -  NFI-00201002.png
gen flagged -  66.09561920166016 file name is -  NFI-00209002.png
gen flagged -  66.09561920166016 file name is -  NFI-00207002.png
gen flagged -  66.09561920166016 file name is -  NFI-00210002.png
gen flagged -  66.09561920166016 file name is -  NFI-00212002.png
gen flagged -  66.09561920166016 file name is -  NFI-00205002.png
gen flagged -  66.09561920166016 file name is -  NFI-00204002.png
gen flagged -  66.09561920166016 file name is -  NFI-00203002.png
gen flagged -  66.09561920166016 file name is -  NFI-00211002.png
gen flagged -  66.09561920166016 file name is -  NFI-00202002.png
When k is  0.008
When k is  0.016
forg passed -  62.801856994628906 file name is -  NFI-06102016.png
forg passed -  62.801856994628906 file name is -  NFI-06106016.png
forg passed -  62.801856994628906 file name is -  NFI-08804016.png
forg passed -  62.801856994628906 file name is -  NFI-07405016.png
forg passed -  62.801